In [1]:
class ECC(object):
    
    def __init__(self, a, b, p, base_point):
        self.curve = (a,b,p)
        self.base_point = base_point
        self.double_base_point = self.double_point(base_point)

    def xgcd(self, b, n):
        x0, x1, y0, y1 = 1, 0, 0, 1
        while n != 0:
            q, b, n = b // n, n, b % n
            x0, x1 = x1, x0 - q * x1
            y0, y1 = y1, y0 - q * y1
        return b, x0, y0

    def inverse(self, b):
        g, x, _ = self.xgcd(b, self.curve[2])
        if g == 1:
            return x % self.curve[2]

    def add_points(self, p, q):
        delta = 0

        if p == None or q == None:
            return p if q == None else q
        elif p[0] == q[0] and p[1] == q[1]:
            delta = (3 * p[0]**2 + self.curve[0]) * \
                    self.inverse(2 * p[1]) % self.curve[2]
        else:
            delta = (p[1] - q[1]) * self.inverse((p[0] - q[0])) % self.curve[2]

        x = (delta * delta - p[0] - q[0]) % self.curve[2]
        y = (delta * (p[0] - x) - p[1]) % self.curve[2]

        return (x,y)
    
    def double_point(self, p, k = 1):
        Q = p
        for i in range(0,k):
            Q = self.add_points(Q,Q)
        return Q
    
    def base_point_mult(self, k):
        Q = None
        for i in [1 if digit == '1' else 0 for digit in bin(k)[2:]]:
            Q = self.double_point(Q)
            if i == 1:
                Q = self.add_points(Q, self.base_point)
                
        return Q

class DiffieHellman(object):

    def __init__(self, elliptic_curve, point_g):
        self.elliptic_curve = elliptic_curve
        self.point_g = point_g

    def generate_public_key(self, private_key):
        if private_key < 0:
            raise ValueError
        return self.elliptic_curve.double_point(self.point_g, private_key)

    def secret_key(self, private_key, public_key):
        return self.elliptic_curve.double_point(public_key, private_key)[0]